<a href="https://colab.research.google.com/github/Tharungovind/GOVINDTHARUN_INFO5731_FALL2024/blob/main/Govind_Tharun_Exersice_03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **INFO5731 In-class Exercise 3**

The purpose of this exercise is to explore various aspects of text analysis, including feature extraction, feature selection, and text similarity ranking.

**Expectations**:
*   Use the provided .*ipynb* document to write your code & respond to the questions. Avoid generating a new file.
*   Write complete answers and run all the cells before submission.
*   Make sure the submission is "clean"; *i.e.*, no unnecessary code cells.
*   Once finished, allow shared rights from top right corner (*see Canvas for details*).

**Total points**: 40

**Deadline**: This in-class exercise is due at the end of Friday, at 11:59 PM.

**Late submissions will have a penalty of 10% of the marks for each day of late submission. , and no requests will be answered. Manage your time accordingly.**

**Please check that the link you submitted can be opened and points to the correct assignment.**

## Question 1 (10 Points)
Describe an interesting **text classification or text mining task** and explain what kind of features might be useful for you to build the machine learning model. List your features and explain why these features might be helpful. You need to list at least five different types of features. **Your dataset must be text.**

In [ ]:
# Your answer here (no code for this question, write down your answer as detail as possible for the above questions):

'''
Please write you answer here:

While building a machine learning model , selection of features is important for getting good performance. Some of the usefultypes of features are Numerical, categorial, textual, time and statistical features.

List of Features
1) Bag of Words-The representation of the text data that counts the frequency of each word in the data regardless grammer.
2) Term Frequency-Inverse Document Frequency-A statistical analysis that evaluates importance of word in data or document relative to its frequency
3) Entity features- Entities such as names, locations & dates extracted from data or text
4) Sentiment Features- That calculates sentimate scores(positive, negative) from the text useful in sentiment analysis tasks
5) Parts of Speech Tags-That indicates whether a word is noun, verb, adjective etc, these can serve as features


Data Set
['The product is good', 'Delivery was too late', 'Good package and delivery']


'''

## Question 2 (10 Points)
Write python code to extract these features you discussed above. You can collect a few sample text data for the feature extraction.

In [13]:
# You code here (Please add comments in the code):

import pandas as pd
from textblob import TextBlob

data = {
    'Text': [
        'The product is Good',
        'Delivery was too late.',
        'Good packaging and delivery.'
    ],
    'Label': ['Positive', 'Negative','Positive']
}

df = pd.DataFrame(data)

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

bow_vectorizer = CountVectorizer()
x_bow = bow_vectorizer.fit_transform(df['Text'])
bow_df = pd.DataFrame(x_bow.toarray(), columns=bow_vectorizer.get_feature_names_out())

print("Bag of Words Representation:")
print(bow_df)

tfidf_vectorizer = TfidfVectorizer()
x_tfidf = tfidf_vectorizer.fit_transform(df['Text'])
tfidf_df = pd.DataFrame(x_tfidf.toarray(), columns=tfidf_vectorizer.get_feature_names_out())

print("TF-IDF Representation:")
print(tfidf_df)

df['Sentiment'] = df['Text'].apply(lambda text: TextBlob(text).sentiment.polarity)

print("Sentiment Analysis:")
print(df[['Text', 'Sentiment']])

Bag of Words Representation:
   and  delivery  good  is  late  packaging  product  the  too  was
0    0         0     1   1     0          0        1    1    0    0
1    0         1     0   0     1          0        0    0    1    1
2    1         1     1   0     0          1        0    0    0    0
TF-IDF Representation:
        and  delivery      good        is      late  packaging   product  \
0  0.000000  0.000000  0.402040  0.528635  0.000000   0.000000  0.528635   
1  0.000000  0.402040  0.000000  0.000000  0.528635   0.000000  0.000000   
2  0.562829  0.428046  0.428046  0.000000  0.000000   0.562829  0.000000   

        the       too       was  
0  0.528635  0.000000  0.000000  
1  0.000000  0.528635  0.528635  
2  0.000000  0.000000  0.000000  
Sentiment Analysis:
                           Text  Sentiment
0           The product is Good        0.7
1        Delivery was too late.       -0.3
2  Good packaging and delivery.        0.7


## Question 3 (10 points):
Use any of the feature selection methods mentioned in this paper "Deng, X., Li, Y., Weng, J., & Zhang, J. (2019). Feature selection for text classification: A review. Multimedia Tools & Applications, 78(3)."

Select the most important features you extracted above, rank the features based on their importance in the descending order.

In [14]:
# You code here (Please add comments in the code):
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import chi2
from sklearn.preprocessing import LabelEncoder


data = {
    'text': ["Climate change is affecting global weather patterns", "The impacts of climate change are profound and far-reaching", "Weather patterns are becoming more unpredictable due to climate change"],
    'label': ['text1', 'text2', 'text3']
}
df = pd.DataFrame(data)

tfidf = TfidfVectorizer()
x_tfidf = tfidf.fit_transform(df['text'])

le = LabelEncoder()
y = le.fit_transform(df['label'])


chi2_values, p_values = chi2(x_tfidf, y)

feature_name = tfidf.get_feature_names_out()
chi2_df = pd.DataFrame({'feature': feature_name, 'chi_rank': chi2_values})

ranked_feature = chi2_df.sort_values(by='chi_rank', ascending=False)

print(ranked_feature)

          feature  chi_rank
0       affecting  0.907737
8          global  0.907737
10             is  0.907737
3        becoming  0.733588
17             to  0.733588
18  unpredictable  0.733588
11           more  0.733588
6             due  0.733588
7             far  0.695214
9         impacts  0.695214
1             and  0.695214
12             of  0.695214
14       profound  0.695214
15       reaching  0.695214
16            the  0.695214
13       patterns  0.322607
19        weather  0.322607
2             are  0.272248
4          change  0.009728
5         climate  0.009728


## Question 4 (10 points):
Write python code to rank the text based on text similarity. Based on the text data you used for question 2, design a query to match the most relevant docments. Please use the BERT model to represent both your query and the text data, then calculate the cosine similarity between the query and each text in your data. Rank the similary with descending order.

In [15]:
# You code here (Please add comments in the code):

import torch
from transformers import BertTokenizer, BertModel
from sklearn.metrics.pairwise import cosine_similarity

data = {
    'Text': [
        'The product is good',
        'Delivery was good.',
        'Good packaging and delivery.'
    ]
}

df = pd.DataFrame(data)

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

def bert_embedding(text):
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=512)
    outputs = model(**inputs)

    cls_embedding = outputs.last_hidden_state[:, 0, :].detach().numpy()
    return cls_embedding

query = "The product is good"
query_embedding = bert_embedding(query)

similarities = []

for text in df['Text']:
    text_embedding = bert_embedding(text)
    similarity = cosine_similarity(query_embedding, text_embedding)[0][0]
    similarities.append(similarity)

df['similarity'] = similarities

df_sorted = df.sort_values(by='similarity', ascending=False)

print(df_sorted[['Text', 'similarity']])


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


                           Text  similarity
0           The product is good    1.000000
1            Delivery was good.    0.909877
2  Good packaging and delivery.    0.844542


# Mandatory Question

**Important: Reflective Feedback on this exercise**

Please provide your thoughts and feedback on the exercises you completed in this assignment. Consider the following points in your response:

Learning Experience: Describe your overall learning experience in working on extracting features from text data. What were the key concepts or techniques you found most beneficial in understanding the process?

Challenges Encountered: Were there specific difficulties in completing this exercise?

Relevance to Your Field of Study: How does this exercise relate to the field of NLP?

**(Your submission will not be graded if this question is left unanswered)**



In [12]:
# Your answer here (no code for this question, write down your answer as detail as possible for the above questions):

'''
Please write you answer here:

Extraction of features from text data it was a intresting topic to learn i really enjoyed in learning and executing the code for this topic, though i got some challenges i referred to lectures and able to solve it


In NLP text classification allows machine to understand and organize the textual data and moreover without text classification wont able to organize the unstructured text data




'''

'\nPlease write you answer here:\n\nExtraction of features from text data it was a intresting topic to learn i really enjoyed in learning and executing the code for this topic, though i got some challenges i referred to lectures and able to solve it\n\n\nIn NLP text classification allows machine to understand and organize the textual data and moreover without text classification wont able to organize the unstructured text data\n\n\n\n\n'